# Worksheet 5 - Scientific Visualization MVE080/MMG640
## Animations in Blender via Python scripting

Name: *Your Name*

This is the fifth worksheet in the course *Scientific Visualization*. It is intended to help us understand *(a)* how animations in Blender work and *(b)* how to control Blender using Python scripting.

Once you're finished with all the tasks, export this document as an HTML-file and upload it in Canvas.
In this work-sheet you are also required to include in material (images and movies) created with Blender. 
To do this, create a code cell and write
```python
%%HTML
<div align="middle">
<video width="80%" controls>
      <source src="path/to/movie.mp4" type="video/mp4">
</video></div>
```
where you of course replace `path/to/movie.mp4` with the path to your movie (if it is in the same folder as your notebook you only need to specify the name).
Execute the cell to show the movie. Make sure the movie is visible in the notebook before you export it to an HTML-file.

You are encouraged to discuss problems and solutions with your fellow students (in the class-room but also on CampusWire), but each student must solve all tasks by themselves and hand-in their own report.
Notice that Jupyter notebooks use [Markdown](https://docs.github.com/en/github/writing-on-github/getting-started-with-writing-and-formatting-on-github/basic-writing-and-formatting-syntax#links) for writing text cells. Make sure you understand the basics. You can also include $\LaTeX$ in your Markdown cells.

## Introduction

To reach the functionality of Blender from Python you have to load the `bpy` module. Your should also load the `math` and `mathutil` modules, as Blender heavily use them.

Blender comes shipped with its own Python version (within Blender). This is to make sure that Python connects properly to the blender software. There are (at least) two ways of using Python for scripting in Blender.

#### Using Python from within Blender

Read about the [Blender Python Console](https://docs.blender.org/manual/en/latest/editors/python_console.html) and the [Text Editor](https://docs.blender.org/manual/en/latest/editors/text_editor.html).
This is the easiest way to run Python commands for Blender. However, I find that the text editor in Blender is not so good, and also it is a bit tricky to debug code using the Blender Python editor and console.

#### Connecting Blender Python to the notebook

Another option is to connect Blender to the notebook. This can be achieved via the [`blender-notebook` module](https://pypi.org/project/blender-notebook/). Note that this is a third party solution, so there's not guarantee that it works (depends on your version of Blender and Python etc). The easiest way to do this is to first create a conda *environment* specifically for Blender. If you open a new terminal, this is how you do it (assuming you're using Blender version 2.93):
```
> conda create --name blender python=3.9.1 numpy scipy matplotlib seaborn
> conda activate blender
```
then install `blender-notebook` is [instructed here](https://pypi.org/project/blender-notebook/) using `pip`. If you then start a notebook
```
> jupyter notebook
```
you can start a Blender Python session by selecting `new -> blender`.

## Task 1: first Blender animation

In this exercise you will create your first, very simple Blender animation.

- First read about [Animation](https://docs.blender.org/manual/en/dev/animation/introduction.html) and [Keyframes](https://docs.blender.org/manual/en/dev/animation/keyframes/introduction.html) in Blender. Also check out the [official video tutorials](https://www.youtube.com/playlist?list=PLa1F2ddGya_-UvuAqHAksYnB0qL9yWDO6) on animation. Start with the 'Keyframes' and 'Timeline' tutorials. You can also watch 'Dopesheet' and 'Graph Editor' if you like.
- Create an object in Blender and place it at the origin (the default box will do, but you can also create something else, for example a doughnut as in the Blender Guru tutorial).
- Open a [Timeline Editor](https://docs.blender.org/manual/en/dev/editors/timeline.html), then add a keyframe at frame 1 by going to the [3D View in Object Mode](https://docs.blender.org/manual/en/dev/editors/3dview/modes.html) and then pressing "I" then "Location". This adds a keyframe for the location of your object.
- Now go to the Timeline Editor and select frame 250. Move your object to some new location (by pressing "G" + mouse move + "Enter"). Insert now another keyframe for the location of your object.
- Play your movie by pressing the play button in the Timeline Editor.
- Render your animation. Make sure that you use MPEG-4 as encoder, and the file extension should be .mp4: in the 'Scene' properties, in the 'Output' pane select File Format 'FFmpeg video', and then under the 'Encoding' pane select 'MPEG-4'. The default output folder in the 'Output' pane is '/tmp/' which probably not what you want: change to a more suitable folder. To create the animation, then choose 'Render Animation' under the 'Render' menu. The default name of the movie created is '0001-0250.mp4', but you can change this.

**NOTE:** Rendering time can be very long, especially on the Linux computers (they seem to be very slow). Thus, it is a good idea to render just a few frames first to make sure it looks good. (Choose 'Render' instead of 'Render Animation' in the 'Render' menu to render single images instead of the whole animation.)

**NOTE:** The movie you get will be rather dull. To spice it up, you can add materials and more advanced lightning to your scene: see [official video guide](https://www.youtube.com/playlist?list=PLa1F2ddGya_-UvuAqHAksYnB0qL9yWDO6) on materials and lightning. Blender Guru also has good tutorials for this. This is not part of this exercise though.

In [ ]:
# Display your movie here (change "path/to/movie.mp4" to the name of the animation you created)
# Adjust the width setting if you like so it fits nicely in the notebook.
%%HTML
<div align="middle">
<video width="80%" controls>
      <source src="path/to/movie.mp4" type="video/mp4">
</video></div>

## Task 2: Blender Animation by Python Scripting

In this exercise you will modify your animation from the previous exercise, by creating the keyframes in a Python script.

The aim is to create an animation that changes the location of your object by the function $\mathbf r(t) = [5 \cos(t), 5 \sin(t), \sin(5t)]$ for $0 \leq t \leq 2*\pi$. 
For this, you will need a little Python scripting. 
That is, you need to write, using the text editor, some Python code that inserts position keyframes according to the function $\mathbf r(t)$. Below is a rough outline of how the code could look like.

**Remember:** you can evaluate this code from within the notebook only if you have installed the `blender-notebook` module, and you're currently running the notebook using the `blender` kernel. Otherwise, you have to copy the code into a script file and run that file from within Blender. Make sure to paste your final code here before you export the notebook to Canvas.

In [36]:
import bpy
from math import *
from mathutils import *

# Variable for currently active object
myobj = bpy.context.object
# Alternatively, if you know that the object is called 'Cube'
# you can reach it by
# myobj = bpy.data.objects['Cube']

# Clear all previous animation data
myobj.animation_data_clear()

# set first and last frame index
total_time = 2*pi # Animation should be 2*pi seconds long
fps = 24 # Frames per second (fps)
bpy.context.scene.frame_start = 0
bpy.context.scene.frame_end = int(total_time*fps)+1

# loop of frames and insert keyframes every 10th frame
keyframe_freq = 10
nlast = bpy.context.scene.frame_end
for n in range(nlast):
    t = total_time*n/nlast

    # Check if n is a multiple of keyframe_freq
    if n%keyframe_freq == 0:
        # Set frame like this
        bpy.context.scene.frame_set(n)

        # Set current location like this
        # myobj.location.x = ...
        # myobj.location.y = ...
        # myobj.location.z = ...

        # Insert new keyframe for "location" like this
        myobj.keyframe_insert(data_path="location")

## Task 3: Rigid body dynamics in Blender by Python scripting 

In this exercise you will visualize the unstable relative equilibrium of a free rigid in Blender by using Python scripting (see [this presentation](https://slides.com/kmodin/mmg640-velocity-16-18?token=eol1jbFe)). 
Create an ellipsoid with half axes $(0.5, 2, 4)$. 
The easiest way is to first create a sphere and then flatten or extrude it differently in the $x$, $y$, and $z$ directions. 
The task is to create a Python script that solves the Euler equations for a rigid body in quaternion formulation using the Lie-Euler method (as explained in the presentation), and from this solution generate rotation keyframes in Blender for the object in your scene. The physically correct way to do this is to first calculate the moments of intertia for your object. However, as this is a little complicated to do in Blender, I give you the moments of inertia tensor: it is diagonal with diagonal elements $(0.5, 2, 4)$. 
This corresponds to your ellipsoid object. Here is the data for the full problem:

- The initial data is $q_0 = (1,0,0,0)$ and $\omega=(10^{-2},1,0)$.
- The total simulation time interval is $[0, 25]$.
- The stepsize is $h = 0.01$.
- Make your simulation so that each frame in Blender corresponds to 10 time steps with the Lie-Euler method.

Here is some additional information that will help you get on track:
- Make sure everything from the `math` and `mathutils` modules are loaded. They support quaternions through the class `Quaternion`. For example, to create and multiply two quaternions you do the following
```python
q1 = Quaternion((1.0, 2.0, 3.0, 4.0))
q2 = Quaternion((0.0, 0.5, 1.5, -0.5))
q3 = q1@q2
```
- To select that Blender should use quaternions for rotations for the object in the variable `myobj`, do
`myobj.rotation_mode = 'QUATERNION'`. After that, to get or set the quaternion representing the orientation of the currently selected object, do 
`q = myobj.rotation_quaternion`, or, to set it, do
`myobj.rotation_quaternion = q`.
- For the `omega` variable (3 vector) it is convenient to use the `Vector` class (also available through `mathutils`). This class also supports the dot and cross products. For example, to create two vectors and take their cross product, do 
```python
omega1 = Vector((1.0, -3.0, 0.0))
omega2 = Vector((0.0, 2.0, -2.0))
omega3 = omega1.cross(omega2)
```
- Likewise, matrices can be created from the `Matrix` class. To construct the diagonal inertia tensor, do 
`I = Matrix(((0.5,0,0),(0,2,0),(0,0,4)))`. Its inverse can be computed with
`I.inverted()`. Multiplication with a `omega` (object from class `Vector`) is obtained as
`I@omega`.
- The `qexp` function, returning the quaternion corresponding to the matrix exponential ([see slides](https://slides.com/kmodin/mmg640-velocity-16-18?token=eol1jbFe)) is obtained by sending in a vector of length 3 (corresponding to `omega`) to the constructor of the `Quaternion` class. For example, if `omeg` is a `Vector` object, then `qexp(omega)` is computed by
`q = Quaternion(omega)`.

Write below your code for creating the animation (either copy your script from Blender, or use it directly here if you're using `blender-notebook` as explained).

In [2]:
# YOUR CODE HERE TO CREATE THE ANIMATION IN BLENDER

When you're finished, render your animation and save it as an `.mp4` file and display it here:

In [ ]:
# Display your movie here (change "path/to/movie.mp4" to the name of the animation you created)
# Adjust the width setting if you like so it fits nicely in the notebook.
%%HTML
<div align="middle">
<video width="80%" controls>
      <source src="path/to/movie.mp4" type="video/mp4">
</video></div>